# TP 5 : Apache Spark pour le Traitement de Données Massives

## Objectifs

Cette séance de travaux pratiques présente Apache Spark, un framework de calcul distribué puissant conçu pour le traitement de données à grande échelle. Vous apprendrez à exploiter les capacités de Spark pour gérer des ensembles de données massifs qui dépassent les limites de mémoire d'une seule machine.

### Objectifs d'apprentissage
* Comprendre l'architecture Spark : Driver, Executors et Cluster Manager
* Maîtriser les opérations RDD (Resilient Distributed Dataset)
* Travailler avec les DataFrames et comprendre la gestion des schémas
* Écrire des requêtes Spark SQL efficaces
* Implémenter des jointures, fonctions fenêtrées et agrégations complexes
* Optimiser les performances grâce aux stratégies de partitionnement et de mise en cache
* Traiter des ensembles de données à grande échelle avec des formats de fichiers en colonnes (Parquet, ORC)

### Prérequis
* Complétion du TP 4 (Calcul Parallèle et Distribué)
* Compréhension des concepts de programmation fonctionnelle (map, filter, reduce)
* Connaissances de base en SQL
* Fondamentaux de la programmation Python

### Installation

Installez PySpark avant de commencer :
```bash
!pip install pyspark==3.5.3
```

### Aperçu des exercices

| Exercice | Sujet | Difficulté |
|----------|-------|------------|
| 1 | Architecture Spark et bases des RDD | ★ |
| 2 | Transformations et actions RDD | ★ |
| 3 | DataFrames et gestion des schémas | ★★ |
| 4 | Spark SQL et requêtes complexes | ★★ |
| 5 | Jointures, fonctions fenêtrées et agrégations | ★★ |
| 6 | Partitionnement, mise en cache et optimisation | ★★★ |
| 7 | Traitement d'ensembles de données à grande échelle | ★★★ |

---

## Exercice 1 : Architecture Spark et bases des RDD [★]

### Comprendre l'architecture Spark

Apache Spark utilise une **architecture maître-esclave** :

1. **Programme Driver** : Le programme principal qui crée le SparkContext et coordonne l'exécution
2. **Cluster Manager** : Alloue les ressources (peut être Standalone, YARN, Mesos ou Kubernetes)
3. **Executors** : Processus de travail qui exécutent les tâches et stockent les données

```
┌─────────────────┐
│  Programme Driver │
│  (SparkContext)   │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ Cluster Manager   │
└────────┬────────┘
         │
    ┌────┴────┐
    ▼         ▼
┌───────┐ ┌───────┐
│Executor│ │Executor│
│ Tâche   │ │ Tâche   │
│ Tâche   │ │ Tâche   │
└───────┘ └───────┘
```

### RDD (Resilient Distributed Dataset)

Les RDD sont la structure de données fondamentale dans Spark :
- **Résilient** : Tolérant aux pannes grâce aux informations de lignée
- **Distribué** : Les données sont partitionnées sur plusieurs nœuds
- **Dataset** : Collection d'éléments pouvant être traités en parallèle

In [ ]:
!pip install pyspark==3.5.3

In [ ]:
# Tout d'abord, vérifions l'installation de PySpark
from pyspark import SparkConf
from pyspark.context import SparkContext

print("PySpark importé avec succès !")

In [ ]:
# Créer un SparkContext avec une configuration locale
# 'local[*]' utilise tous les cœurs disponibles
conf = SparkConf().setAppName("TP5").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

# Afficher la configuration Spark
print(f"Version Spark : {sc.version}")
print(f"Nom de l'application : {sc.appName}")
print(f"Master : {sc.master}")
print(f"Parallélisme par défaut : {sc.defaultParallelism}")

In [ ]:
# Création de RDD - Méthode 1 : À partir d'une collection Python (parallelize)
nombres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
nombres_rdd = sc.parallelize(nombres)

print(f"Type : {type(nombres_rdd)}")
print(f"Nombre de partitions : {nombres_rdd.getNumPartitions()}")
print(f"Premier élément : {nombres_rdd.first()}")
print(f"Tous les éléments : {nombres_rdd.collect()}")

In [ ]:
# Création de RDD - Méthode 2 : À partir de fichiers externes
lignes_rdd = sc.textFile("../shared_data/pl.csv")

print(f"Nombre de partitions : {lignes_rdd.getNumPartitions()}")
print(f"Nombre de lignes : {lignes_rdd.count()}")
print(f"\n5 premières lignes :")
for ligne in lignes_rdd.take(5):
    print(f"  {ligne}")

In [ ]:
# Création de RDD avec un nombre spécifique de partitions
# Plus de partitions = plus de parallélisme (mais aussi plus de surcharge)
nombres_4partitions = sc.parallelize(range(1, 101), numSlices=4)
nombres_8partitions = sc.parallelize(range(1, 101), numSlices=8)

print(f"RDD avec 4 partitions : {nombres_4partitions.getNumPartitions()}")
print(f"RDD avec 8 partitions : {nombres_8partitions.getNumPartitions()}")

# Voir comment les données sont distribuées entre les partitions
print(f"\nDistribution des données (4 partitions) :")
print(nombres_4partitions.glom().collect())

### Questions - Exercice 1

**Q1.1** Créez un RDD contenant les carrés des nombres de 1 à 1000. Expérimentez avec différents nombres de partitions (2, 4, 8, 16) et expliquez comment le partitionnement affecte la distribution des données.

**Q1.2** Chargez plusieurs fichiers CSV depuis le répertoire `../shared_data/` en utilisant des motifs génériques (ex: `*.csv`). Combien y a-t-il de lignes au total dans tous les fichiers ?

**Q1.3** Expliquez la différence entre la création d'un RDD avec `parallelize()` et `textFile()`. Quand utiliseriez-vous chaque méthode ?

In [ ]:
# Vos solutions ici


---

## Exercice 2 : Transformations et actions RDD [★]

### Transformations vs Actions

Les opérations Spark sont divisées en deux catégories :

**Transformations** (Paresseuses - créent de nouveaux RDD) :
- `map()`, `filter()`, `flatMap()`, `distinct()`
- `union()`, `intersection()`, `subtract()`
- `groupByKey()`, `reduceByKey()`, `sortByKey()`

**Actions** (Immédiates - retournent des résultats) :
- `collect()`, `count()`, `first()`, `take()`
- `reduce()`, `fold()`, `aggregate()`
- `saveAsTextFile()`, `foreach()`

### Évaluation paresseuse

Les transformations sont **paresseuses** : Spark construit un DAG (Graphe Acyclique Dirigé) des opérations mais ne les exécute pas jusqu'à ce qu'une action soit appelée. Cela permet à Spark d'optimiser le plan d'exécution.

In [ ]:
# Démonstration de l'évaluation paresseuse
import time

# Cette transformation n'est PAS exécutée immédiatement
debut = time.time()
grand_rdd = sc.parallelize(range(1, 1000001))
carres = grand_rdd.map(lambda x: x ** 2)
filtres = carres.filter(lambda x: x % 2 == 0)
print(f"Transformations définies en : {time.time() - debut:.4f} secondes")

# L'action déclenche l'exécution
debut = time.time()
resultat = filtres.count()
print(f"Action exécutée en : {time.time() - debut:.4f} secondes")
print(f"Nombre de carrés pairs : {resultat}")

In [ ]:
# Transformation map : appliquer une fonction à chaque élément
mots = sc.parallelize(["bonjour", "monde", "spark", "python"])

# Transformer en majuscules
mots_majuscules = mots.map(lambda m: m.upper())
print(f"Majuscules : {mots_majuscules.collect()}")

# Transformer en tuples (mot, longueur)
longueurs_mots = mots.map(lambda m: (m, len(m)))
print(f"Longueurs des mots : {longueurs_mots.collect()}")

In [ ]:
# Transformation filter : garder les éléments qui satisfont une condition
nombres = sc.parallelize(range(1, 21))

# Garder uniquement les nombres pairs
pairs = nombres.filter(lambda x: x % 2 == 0)
print(f"Nombres pairs : {pairs.collect()}")

# Garder les nombres divisibles par 3
div_par_3 = nombres.filter(lambda x: x % 3 == 0)
print(f"Divisibles par 3 : {div_par_3.collect()}")

In [ ]:
# Transformation flatMap : map qui peut retourner plusieurs éléments
phrases = sc.parallelize([
    "Bonjour le Monde",
    "Apache Spark est puissant",
    "Traitement de données massives"
])

# Découper les phrases en mots
mots = phrases.flatMap(lambda s: s.split())
print(f"Tous les mots : {mots.collect()}")
print(f"Nombre de mots : {mots.count()}")

In [ ]:
# Action reduce : agréger les éléments avec une fonction
nombres = sc.parallelize([1, 2, 3, 4, 5])

# Somme de tous les nombres
total = nombres.reduce(lambda a, b: a + b)
print(f"Somme : {total}")

# Trouver le maximum

maximum = numbers.reduce(lambda a, b: a if a > b else b)
print(f"Maximum : {maximum}")

# Produit de tous les nombres
produit = nombres.reduce(lambda a, b: a * b)
print(f"Produit : {produit}")

In [ ]:
# Comptage de mots - L'exemple classique de Spark
texte = sc.parallelize([
    "Apache Spark est un moteur d'analytique unifié",
    "Spark fournit des API de haut niveau en Java Scala Python et R",
    "Spark alimente une pile de bibliothèques pour SQL streaming et apprentissage automatique",
    "Spark fonctionne sur Hadoop YARN Mesos Kubernetes et en mode autonome"
])

# Pipeline de comptage de mots
comptage_mots = (texte
    .flatMap(lambda ligne: ligne.lower().split())  # Découper en mots
    .map(lambda mot: (mot, 1))                      # Mapper en paires (mot, 1)
    .reduceByKey(lambda a, b: a + b)                # Sommer les comptages par mot
    .sortBy(lambda x: x[1], ascending=False))       # Trier par comptage

print("Comptage de mots (top 10) :")
for mot, comptage in comptage_mots.take(10):
    print(f"  {mot}: {comptage}")

In [ ]:
# Opérations ensemblistes sur les RDD
rdd1 = sc.parallelize([1, 2, 3, 4, 5])
rdd2 = sc.parallelize([4, 5, 6, 7, 8])

# Union (tous les éléments des deux)
print(f"Union : {rdd1.union(rdd2).collect()}")

# Intersection (éléments communs)
print(f"Intersection : {rdd1.intersection(rdd2).collect()}")

# Soustraction (éléments dans rdd1 mais pas dans rdd2)
print(f"Soustraction : {rdd1.subtract(rdd2).collect()}")

# Distinct (éléments uniques)
print(f"Union distincte : {rdd1.union(rdd2).distinct().collect()}")

### Questions - Exercice 2

**Q2.1** Chargez le fichier `../shared_data/pl.csv` et effectuez les opérations suivantes :
- Comptez le nombre total de caractères sur toutes les lignes
- Comptez le nombre total de tokens (valeurs séparées par des virgules)
- Trouvez la ligne avec le nombre maximum de caractères

**Q2.2** Téléchargez 50 pages HTML depuis le web. Écrivez un programme Spark pour :
- Compter le nombre total de balises `<div>` et `</div>` dans tous les fichiers
- Trouver la page avec le plus de balises `<a>` (ancres)
- Extraire et compter tous les noms de classes CSS uniques

**Q2.3** Implémentez un compteur de fréquence de caractères qui :
- Lit tous les fichiers texte d'un répertoire
- Compte la fréquence de chaque caractère (insensible à la casse)
- Retourne les 10 caractères les plus fréquents

In [ ]:
# Vos solutions ici


---

## Exercice 3 : DataFrames et gestion des schémas [★★]

### DataFrames

Les DataFrames sont une abstraction de plus haut niveau construite au-dessus des RDD :
- Organisés en colonnes nommées (comme une table)
- Optimisés grâce à l'optimiseur de requêtes Catalyst
- Support pour les données structurées et semi-structurées
- Meilleures performances que les RDD pour la plupart des opérations

### SparkSession

SparkSession est le point d'entrée pour les opérations sur les DataFrames (introduit dans Spark 2.0).

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType

# Créer une SparkSession
spark = SparkSession.builder \
    .appName("TP5-DataFrames") \
    .getOrCreate()

print(f"Session Spark créée : {spark.version}")

In [ ]:
# Création de DataFrames - Méthode 1 : À partir d'une liste Python avec schéma inféré
donnees = [
    ("Python", 1991, "Guido van Rossum"),
    ("Java", 1995, "James Gosling"),
    ("JavaScript", 1995, "Brendan Eich"),
    ("C", 1972, "Dennis Ritchie"),
    ("Rust", 2010, "Graydon Hoare")
]

df_infere = spark.createDataFrame(donnees, ["langage", "annee", "createur"])
df_infere.show()
df_infere.printSchema()

In [ ]:
# Création de DataFrames - Méthode 2 : Avec schéma explicite
schema = StructType([
    StructField("langage", StringType(), nullable=False),
    StructField("annee", IntegerType(), nullable=False),
    StructField("createur", StringType(), nullable=True)
])

df_explicite = spark.createDataFrame(donnees, schema)
df_explicite.show()
df_explicite.printSchema()

In [ ]:
# Création de DataFrames - Méthode 3 : À partir d'un fichier JSON
df_json = spark.read.json("../shared_data/pl.json")
df_json.show(10)
df_json.printSchema()

In [ ]:
# Création de DataFrames - Méthode 4 : À partir d'un CSV avec options
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../shared_data/pl.csv")

df_csv.show(10)
df_csv.printSchema()

In [ ]:
# Opérations de base sur les DataFrames
df = df_json

# Afficher les n premières lignes
print("5 premières lignes :")
df.show(5)

# Obtenir les noms des colonnes
print(f"\nColonnes : {df.columns}")

# Obtenir le nombre de lignes et colonnes
print(f"Dimensions : ({df.count()}, {len(df.columns)})")

# Statistiques descriptives
print("\nStatistiques :")
df.describe().show()

In [ ]:
# Sélection et manipulation de colonnes
from pyspark.sql.functions import col, lit, when, upper, lower, length

# Sélectionner des colonnes spécifiques
df.select("languageLabel").show(5)

# Sélectionner avec des expressions de colonnes
df.select(
    col("languageLabel"),
    col("year"),
    (col("year") - 1900).alias("annees_depuis_1900")
).show(5)

In [ ]:
# Ajout et modification de colonnes
df_modifie = df \
    .withColumn("siecle", ((col("year") / 100) + 1).cast(IntegerType())) \
    .withColumn("langage_majuscules", upper(col("languageLabel"))) \
    .withColumn("longueur_nom", length(col("languageLabel")))

df_modifie.show(10)

In [ ]:
# Filtrage des lignes
# Langages créés après 2000
recents = df.filter(col("year") > 2000)
print(f"Langages après 2000 : {recents.count()}")
recents.show(10)

# Conditions multiples
filtres = df.filter((col("year") >= 1990) & (col("year") <= 2000))
print(f"\nLangages de 1990-2000 : {filtres.count()}")
filtres.show()

In [ ]:
# GroupBy et agrégations
from pyspark.sql.functions import count, avg, min, max, sum

# Compter les langages par année
df.groupBy("year") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10)

In [ ]:
# Agrégations multiples
df_modifie.groupBy("siecle") \
    .agg(
        count("*").alias("nb_langages"),
        min("year").alias("annee_debut"),
        max("year").alias("annee_fin"),
        avg("longueur_nom").alias("longueur_moyenne_nom")
    ) \
    .orderBy("siecle") \
    .show()

### Questions - Exercice 3

**Q3.1** Interrogez Wikidata pour télécharger des informations sur toutes les applications logicielles incluant : nom, date de sortie, développeur et langage de programmation utilisé. Chargez ces données dans un DataFrame Spark avec un schéma explicite.

**Q3.2** En utilisant le DataFrame des langages de programmation :
- Ajoutez une colonne catégorisant les langages comme "Pionnier" (avant 1980), "Classique" (1980-2000) ou "Moderne" (après 2000)
- Calculez les statistiques (comptage, année min, année max) pour chaque catégorie
- Trouvez la décennie avec le plus de sorties de langages

**Q3.3** Créez un DataFrame à partir d'un fichier CSV avec une gestion correcte de :
- Valeurs manquantes (nulls)
- Analyse des dates
- Délimiteurs personnalisés
- Caractères échappés

In [ ]:
# Vos solutions ici


---

## Exercice 4 : Spark SQL et requêtes complexes [★★]

### Interface SQL

Spark SQL vous permet d'exécuter des requêtes SQL directement sur les DataFrames en utilisant des vues temporaires. C'est utile pour :
- Les requêtes complexes plus faciles à exprimer en SQL
- L'interopérabilité avec les outils basés sur SQL
- La familiarité pour les utilisateurs SQL

In [ ]:
# Charger les données et créer une vue temporaire
df = spark.read.json("../shared_data/pl.json")
df.createOrReplaceTempView("langages")

# Requête SELECT de base
spark.sql("SELECT * FROM langages LIMIT 10").show()

In [ ]:
# Filtrage avec la clause WHERE
spark.sql("""
    SELECT languageLabel, year 
    FROM langages 
    WHERE year >= 1990 AND year <= 2000
    ORDER BY year
""").show()

In [ ]:
# Requêtes d'agrégation
spark.sql("""
    SELECT 
        year,
        COUNT(*) as nombre_langages
    FROM langages
    GROUP BY year
    HAVING COUNT(*) > 3
    ORDER BY nombre_langages DESC
""").show(10)

In [ ]:
# Utilisation des expressions CASE
spark.sql("""
    SELECT 
        languageLabel,
        year,
        CASE 
            WHEN year < 1980 THEN 'Ere Pionniere'
            WHEN year < 1990 THEN 'Annees 1980'
            WHEN year < 2000 THEN 'Annees 1990'
            WHEN year < 2010 THEN 'Annees 2000'
            ELSE 'Annees 2010+'
        END as epoque
    FROM langages
    ORDER BY year
""").show(20)

In [ ]:
# Sous-requêtes
spark.sql("""
    SELECT languageLabel, year
    FROM langages
    WHERE year = (
        SELECT MAX(year) FROM langages
    )
""").show()

In [ ]:
# Expressions de table communes (CTE)
spark.sql("""
    WITH stats_decennie AS (
        SELECT 
            FLOOR(year / 10) * 10 as decennie,
            COUNT(*) as comptage
        FROM langages
        GROUP BY FLOOR(year / 10) * 10
    )
    SELECT 
        decennie,
        comptage,
        ROUND(comptage * 100.0 / SUM(comptage) OVER(), 2) as pourcentage
    FROM stats_decennie
    ORDER BY decennie
""").show()

In [ ]:
# Créer des tables supplémentaires pour les exemples de jointures
# Données de paradigmes
donnees_paradigmes = [
    ("Python", "Multi-paradigme"),
    ("Java", "Orienté objet"),
    ("JavaScript", "Multi-paradigme"),
    ("Haskell", "Fonctionnel"),
    ("C", "Procédural"),
    ("Rust", "Multi-paradigme"),
    ("Lisp", "Fonctionnel"),
    ("Prolog", "Logique")
]

paradigmes_df = spark.createDataFrame(donnees_paradigmes, ["langage", "paradigme"])
paradigmes_df.createOrReplaceTempView("paradigmes")

# Données de typage
donnees_typage = [
    ("Python", "Dynamique", "Fort"),
    ("Java", "Statique", "Fort"),
    ("JavaScript", "Dynamique", "Faible"),
    ("C", "Statique", "Faible"),
    ("Rust", "Statique", "Fort"),
    ("Haskell", "Statique", "Fort")
]

typage_df = spark.createDataFrame(donnees_typage, ["langage", "typage", "surete_type"])
typage_df.createOrReplaceTempView("typage")

print("Paradigmes :")
paradigmes_df.show()
print("Typage :")
typage_df.show()

In [ ]:
# INNER JOIN
spark.sql("""
    SELECT p.langage, p.paradigme, t.typage, t.surete_type
    FROM paradigmes p
    INNER JOIN typage t ON p.langage = t.langage
""").show()

In [ ]:
# LEFT JOIN
spark.sql("""
    SELECT p.langage, p.paradigme, t.typage
    FROM paradigmes p
    LEFT JOIN typage t ON p.langage = t.langage
""").show()

### Questions - Exercice 4

**Q4.1** En utilisant la vue langages, écrivez des requêtes SQL pour :
- Trouver tous les langages sortis la même année que Python (1991)
- Calculer le nombre moyen de langages sortis par décennie
- Trouver les années où plus de 5 langages sont sortis

**Q4.2** Créez deux nouvelles vues à partir de Wikidata :
- Applications logicielles avec leurs développeurs
- Développeurs avec leurs pays
Écrivez une requête joignant ces tables pour afficher les logiciels groupés par pays.

**Q4.3** En utilisant les fonctions fenêtrées, écrivez des requêtes pour :
- Classer les langages par année au sein de chaque décennie
- Calculer le total cumulatif des langages sortis au fil du temps
- Trouver le premier et le dernier langage sorti chaque décennie

In [ ]:
# Vos solutions ici


---

## Exercice 5 : Jointures, fonctions fenêtrées et agrégations [★★]

### Opérations avancées sur les DataFrames

Cet exercice couvre des opérations plus complexes essentielles pour le traitement de données réel :
- Différents types de jointures
- Fonctions fenêtrées pour l'analytique
- Agrégations complexes

In [ ]:
from pyspark.sql.functions import col, row_number, rank, dense_rank, lag, lead, sum as spark_sum
from pyspark.sql.window import Window

# Créer des données de ventes exemple
donnees_ventes = [
    ("2024-01-15", "Electronique", "Ordinateur", 1200.00, 5),
    ("2024-01-15", "Electronique", "Telephone", 800.00, 10),
    ("2024-01-16", "Electronique", "Ordinateur", 1200.00, 3),
    ("2024-01-16", "Vetements", "Chemise", 50.00, 20),
    ("2024-01-17", "Electronique", "Tablette", 500.00, 8),
    ("2024-01-17", "Vetements", "Pantalon", 80.00, 15),
    ("2024-01-18", "Vetements", "Chemise", 50.00, 25),
    ("2024-01-18", "Electronique", "Telephone", 800.00, 12),
    ("2024-01-19", "Livres", "Fiction", 25.00, 30),
    ("2024-01-19", "Livres", "Technique", 60.00, 10),
]

schema_ventes = ["date", "categorie", "produit", "prix", "quantite"]
ventes_df = spark.createDataFrame(donnees_ventes, schema_ventes)

# Ajouter une colonne calculée
ventes_df = ventes_df.withColumn("chiffre_affaires", col("prix") * col("quantite"))
ventes_df.show()

In [ ]:
# Fonction fenêtrée : Numéro de ligne dans chaque catégorie
spec_fenetre = Window.partitionBy("categorie").orderBy(col("chiffre_affaires").desc())

ventes_classees = ventes_df.withColumn("rang_dans_categorie", row_number().over(spec_fenetre))
ventes_classees.show()

In [ ]:
# Différentes fonctions de classement
spec_fenetre = Window.partitionBy("categorie").orderBy(col("chiffre_affaires").desc())

ventes_df.select(
    "categorie",
    "produit",
    "chiffre_affaires",
    row_number().over(spec_fenetre).alias("numero_ligne"),
    rank().over(spec_fenetre).alias("rang"),
    dense_rank().over(spec_fenetre).alias("rang_dense")
).show()

In [ ]:
# Totaux cumulatifs avec les fonctions fenêtrées
fenetre_cumul = Window.partitionBy("categorie").orderBy("date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

ventes_df.select(
    "date",
    "categorie",
    "chiffre_affaires",
    spark_sum("chiffre_affaires").over(fenetre_cumul).alias("total_cumulatif")
).orderBy("categorie", "date").show()

In [ ]:
# Fonctions lag et lead (valeurs de la ligne précédente/suivante)
fenetre_ordonnee = Window.partitionBy("categorie").orderBy("date")

ventes_df.select(
    "date",
    "categorie",
    "chiffre_affaires",
    lag("chiffre_affaires", 1).over(fenetre_ordonnee).alias("ca_precedent"),
    lead("chiffre_affaires", 1).over(fenetre_ordonnee).alias("ca_suivant")
).orderBy("categorie", "date").show()

In [ ]:
# Agrégations complexes avec pivot
pivot_df = ventes_df.groupBy("date").pivot("categorie").sum("chiffre_affaires")
pivot_df.show()

In [ ]:
# Agrégations multiples en une fois
from pyspark.sql.functions import avg, min, max, count, round as spark_round

ventes_df.groupBy("categorie").agg(
    count("*").alias("nb_transactions"),
    spark_round(avg("chiffre_affaires"), 2).alias("ca_moyen"),
    spark_round(min("chiffre_affaires"), 2).alias("ca_min"),
    spark_round(max("chiffre_affaires"), 2).alias("ca_max"),
    spark_round(spark_sum("chiffre_affaires"), 2).alias("ca_total")
).show()

In [ ]:
# Créer des DataFrames supplémentaires pour les exemples de jointures
# Données clients
donnees_clients = [
    (1, "Alice", "Paris"),
    (2, "Bob", "Londres"),
    (3, "Charlie", "Berlin"),
    (4, "Diana", "Madrid")
]
clients_df = spark.createDataFrame(donnees_clients, ["client_id", "nom", "ville"])

# Données commandes
donnees_commandes = [
    (101, 1, "2024-01-15", 150.00),
    (102, 1, "2024-01-16", 200.00),
    (103, 2, "2024-01-15", 300.00),
    (104, 3, "2024-01-17", 450.00),
    (105, 5, "2024-01-18", 100.00)  # Le client 5 n'existe pas
]
commandes_df = spark.createDataFrame(donnees_commandes, ["commande_id", "client_id", "date_commande", "montant"])

print("Clients :")
clients_df.show()
print("Commandes :")
commandes_df.show()

In [ ]:
# Différents types de jointures

# Jointure interne (uniquement les lignes correspondantes)
print("INNER JOIN :")
clients_df.join(commandes_df, "client_id", "inner").show()

# Jointure gauche (tous les clients, commandes correspondantes)
print("LEFT JOIN :")
clients_df.join(commandes_df, "client_id", "left").show()

# Jointure droite (toutes les commandes, clients correspondants)
print("RIGHT JOIN :")
clients_df.join(commandes_df, "client_id", "right").show()

# Jointure externe complète (toutes les lignes des deux côtés)
print("FULL OUTER JOIN :")
clients_df.join(commandes_df, "client_id", "outer").show()

In [ ]:
# Anti-jointure (lignes de gauche sans correspondance à droite)
print("Clients sans commandes (LEFT ANTI) :")
clients_df.join(commandes_df, "client_id", "left_anti").show()

# Semi-jointure (lignes de gauche avec correspondance à droite, sans colonnes de droite)
print("Clients avec commandes (LEFT SEMI) :")
clients_df.join(commandes_df, "client_id", "left_semi").show()

### Questions - Exercice 5

**Q5.1** En utilisant les données de ventes, calculez :
- Le pourcentage du chiffre d'affaires total contribué par chaque catégorie
- Le produit le plus vendu dans chaque catégorie
- Le taux de croissance journalier du chiffre d'affaires (variation en pourcentage par rapport au jour précédent)

**Q5.2** Créez une segmentation client basée sur leurs dépenses totales :
- "Bronze" : dépenses totales < 200
- "Argent" : dépenses totales 200-500
- "Or" : dépenses totales > 500
Affichez le nombre de clients dans chaque segment.

**Q5.3** Téléchargez des données depuis Wikidata sur :
- Les pays et leurs populations
- Les villes et leurs pays
- Les universités et leurs villes
Effectuez des jointures pour trouver les 10 pays avec le plus d'universités, normalisé par la population.

In [ ]:
# Vos solutions ici


---

## Exercice 6 : Partitionnement, mise en cache et optimisation [★★★]

### Optimisation des performances

Les performances de Spark dépendent fortement de :
1. **Partitionnement** : Comment les données sont distribuées entre les nœuds
2. **Mise en cache** : Garder les données fréquemment accédées en mémoire
3. **Éviter les shuffles** : Minimiser les mouvements de données entre les nœuds
4. **Variables broadcast** : Partager efficacement les petites données entre les nœuds

In [ ]:
# Comprendre les partitions
grand_df = spark.range(1000000)  # 1 million de lignes

print(f"Partitions par défaut : {grand_df.rdd.getNumPartitions()}")

# Repartitionner à un nombre spécifique
repartitionne = grand_df.repartition(8)
print(f"Après repartition(8) : {repartitionne.rdd.getNumPartitions()}")

# Coalesce (réduire les partitions sans shuffle complet)
fusionne = grand_df.coalesce(4)
print(f"Après coalesce(4) : {fusionne.rdd.getNumPartitions()}")

In [ ]:
# Partitionner par colonne (utile pour le filtrage)
ventes_partitionnees = ventes_df.repartition(4, "categorie")
print(f"Partitions : {ventes_partitionnees.rdd.getNumPartitions()}")

# Voir le contenu des partitions
def afficher_info_partitions(df):
    partitions = df.rdd.glom().collect()
    for i, partition in enumerate(partitions):
        print(f"Partition {i} : {len(partition)} lignes")

afficher_info_partitions(ventes_partitionnees)

In [ ]:
# Mise en cache des DataFrames
from pyspark import StorageLevel

# Créer un grand DataFrame
grand_df = spark.range(100000).withColumn("carre", col("id") ** 2)

# Sans mise en cache - calculs multiples
import time

debut = time.time()
comptage1 = grand_df.filter(col("carre") > 1000).count()
comptage2 = grand_df.filter(col("carre") > 2000).count()
comptage3 = grand_df.filter(col("carre") > 3000).count()
print(f"Sans mise en cache : {time.time() - debut:.4f}s")

# Avec mise en cache
grand_df.cache()  # ou grand_df.persist(StorageLevel.MEMORY_ONLY)

# La première action déclenche la mise en cache
_ = grand_df.count()

debut = time.time()
comptage1 = grand_df.filter(col("carre") > 1000).count()
comptage2 = grand_df.filter(col("carre") > 2000).count()
comptage3 = grand_df.filter(col("carre") > 3000).count()
print(f"Avec mise en cache : {time.time() - debut:.4f}s")

# Libérer le cache quand terminé
grand_df.unpersist()

In [ ]:
# Niveaux de stockage
print("Niveaux de stockage disponibles :")
print(f"  MEMORY_ONLY : {StorageLevel.MEMORY_ONLY}")
print(f"  MEMORY_AND_DISK : {StorageLevel.MEMORY_AND_DISK}")
print(f"  DISK_ONLY : {StorageLevel.DISK_ONLY}")
print(f"  MEMORY_ONLY_SER : {StorageLevel.MEMORY_ONLY_SER}")
print(f"  MEMORY_AND_DISK_SER : {StorageLevel.MEMORY_AND_DISK_SER}")

In [ ]:
# Variables broadcast pour les petits ensembles de données dans les jointures
from pyspark.sql.functions import broadcast

# Petite table de référence
donnees_ref = [("A", "Catégorie A"), ("B", "Catégorie B"), ("C", "Catégorie C")]
ref_df = spark.createDataFrame(donnees_ref, ["code", "description"])

# Grande table de faits
donnees_faits = [(i, ["A", "B", "C"][i % 3], i * 10) for i in range(10000)]
faits_df = spark.createDataFrame(donnees_faits, ["id", "code", "valeur"])

# Jointure normale
debut = time.time()
resultat1 = faits_df.join(ref_df, "code")
_ = resultat1.count()
print(f"Jointure normale : {time.time() - debut:.4f}s")

# Jointure broadcast (la petite table est diffusée à tous les nœuds)
debut = time.time()
resultat2 = faits_df.join(broadcast(ref_df), "code")
_ = resultat2.count()
print(f"Jointure broadcast : {time.time() - debut:.4f}s")

In [ ]:
# Expliquer le plan d'exécution de la requête
df = ventes_df.filter(col("chiffre_affaires") > 100).groupBy("categorie").sum("chiffre_affaires")

# Explication simple
print("=== Explication simple ===")
df.explain()

# Explication étendue
print("\n=== Explication étendue ===")
df.explain(extended=True)

In [ ]:
# Démonstration du predicate pushdown
# Lors de la lecture de fichiers, Spark peut pousser les filtres vers la source de données

# Écrire les données exemple en parquet
ventes_df.write.mode("overwrite").parquet("donnees_ventes.parquet")

# Lecture avec filtre - Spark ne lira que les données nécessaires
filtre = spark.read.parquet("donnees_ventes.parquet").filter(col("categorie") == "Electronique")
print("Plan de requête avec predicate pushdown :")
filtre.explain()

In [ ]:
# Élagage de colonnes - ne lire que les colonnes nécessaires
# Sélectionner des colonnes spécifiques avant d'appliquer des transformations

# Inefficace : lit toutes les colonnes
toutes_cols = spark.read.parquet("donnees_ventes.parquet")
resultat_tout = toutes_cols.filter(col("chiffre_affaires") > 1000).select("produit", "chiffre_affaires")

# Efficace : ne lit que les colonnes nécessaires
selectionnees = spark.read.parquet("donnees_ventes.parquet").select("produit", "chiffre_affaires")
resultat_select = selectionnees.filter(col("chiffre_affaires") > 1000)

print("Les deux approches produisent le même résultat :")
resultat_tout.show()
resultat_select.show()

### Questions - Exercice 6

**Q6.1** Créez un DataFrame avec 10 millions de lignes. Comparez les performances de :
- Exécuter la même agrégation 5 fois sans mise en cache
- L'exécuter 5 fois avec mise en cache
- Utiliser différents niveaux de stockage (MEMORY_ONLY vs MEMORY_AND_DISK)

**Q6.2** Démontrez l'impact du partitionnement sur les performances des jointures :
- Créez deux grands DataFrames (1 million de lignes chacun)
- Joignez-les avec différentes stratégies de partitionnement
- Comparez les temps d'exécution et expliquez les différences

**Q6.3** Analysez les plans de requêtes :
- Écrivez une requête complexe avec filtres, jointures et agrégations
- Utilisez `explain()` pour comprendre le plan d'exécution
- Optimisez la requête en fonction de l'analyse du plan
- Comparez les performances avant/après

In [ ]:
# Vos solutions ici


---

## Exercice 7 : Traitement d'ensembles de données à grande échelle [★★★]

### Formats de fichiers en colonnes

Pour le traitement de données à grande échelle, les formats en colonnes comme Parquet et ORC offrent des avantages significatifs :
- **Compression efficace** : Les valeurs similaires stockées ensemble se compressent mieux
- **Élagage de colonnes** : Ne lire que les colonnes nécessaires
- **Predicate pushdown** : Filtrer au niveau du stockage
- **Évolution du schéma** : Ajouter/supprimer des colonnes sans réécrire les données

In [ ]:
# Générer un plus grand ensemble de données pour la démonstration
import random
from pyspark.sql.functions import rand, randn, floor, concat, lit

# Créer un plus grand ensemble de données de ventes
grandes_ventes = spark.range(100000) \
    .withColumn("date", concat(lit("2024-"), 
                               ((floor(rand() * 12) + 1).cast("string")), 
                               lit("-"), 
                               ((floor(rand() * 28) + 1).cast("string")))) \
    .withColumn("categorie", 
                when(rand() < 0.3, "Electronique")
                .when(rand() < 0.6, "Vetements")
                .otherwise("Livres")) \
    .withColumn("prix", floor(rand() * 1000) + 10) \
    .withColumn("quantite", floor(rand() * 50) + 1) \
    .withColumn("chiffre_affaires", col("prix") * col("quantite"))

print(f"{grandes_ventes.count()} lignes générées")
grandes_ventes.show(5)

In [ ]:
# Écrire dans différents formats et comparer les tailles
import os

# Écrire en CSV
grandes_ventes.write.mode("overwrite").option("header", "true").csv("grandes_ventes_csv")

# Écrire en Parquet (compression par défaut : snappy)
grandes_ventes.write.mode("overwrite").parquet("grandes_ventes_parquet")

# Écrire en Parquet avec compression gzip
grandes_ventes.write.mode("overwrite").option("compression", "gzip").parquet("grandes_ventes_parquet_gzip")

# Écrire en ORC
grandes_ventes.write.mode("overwrite").orc("grandes_ventes_orc")

In [ ]:
# Comparer les tailles de fichiers (simplifié - l'implémentation réelle dépend du système de fichiers)
def obtenir_taille_dossier(chemin):
    """Calculer la taille totale des fichiers dans un dossier"""
    total = 0
    if os.path.exists(chemin):
        for racine, dossiers, fichiers in os.walk(chemin):
            for f in fichiers:
                total += os.path.getsize(os.path.join(racine, f))
    return total

taille_csv = obtenir_taille_dossier("grandes_ventes_csv")
taille_parquet = obtenir_taille_dossier("grandes_ventes_parquet")
taille_parquet_gzip = obtenir_taille_dossier("grandes_ventes_parquet_gzip")
taille_orc = obtenir_taille_dossier("grandes_ventes_orc")

print(f"Taille CSV : {taille_csv / 1024:.2f} Ko")
print(f"Taille Parquet (snappy) : {taille_parquet / 1024:.2f} Ko")
print(f"Taille Parquet (gzip) : {taille_parquet_gzip / 1024:.2f} Ko")
print(f"Taille ORC : {taille_orc / 1024:.2f} Ko")

if taille_csv > 0:
    print(f"\nRatios de compression vs CSV :")
    print(f"  Parquet (snappy) : {taille_csv / taille_parquet:.2f}x")
    print(f"  Parquet (gzip) : {taille_csv / taille_parquet_gzip:.2f}x")
    print(f"  ORC : {taille_csv / taille_orc:.2f}x")

In [ ]:
# Comparer les performances de lecture
import time

def benchmark_lecture(chemin, type_format):
    debut = time.time()
    if type_format == "csv":
        df = spark.read.option("header", "true").option("inferSchema", "true").csv(chemin)
    elif type_format == "parquet":
        df = spark.read.parquet(chemin)
    else:
        df = spark.read.orc(chemin)
    comptage = df.count()
    return time.time() - debut

print("Performances de lecture (secondes) :")
print(f"  CSV : {benchmark_lecture('grandes_ventes_csv', 'csv'):.4f}s")
print(f"  Parquet : {benchmark_lecture('grandes_ventes_parquet', 'parquet'):.4f}s")
print(f"  ORC : {benchmark_lecture('grandes_ventes_orc', 'orc'):.4f}s")

In [ ]:
# Écritures partitionnées - partitionner par catégorie
grandes_ventes.write.mode("overwrite").partitionBy("categorie").parquet("grandes_ventes_partitionne")

# Lister les répertoires de partition
for element in os.listdir("grandes_ventes_partitionne"):
    if os.path.isdir(os.path.join("grandes_ventes_partitionne", element)):
        print(f"Partition : {element}")

In [ ]:
# Lecture de données partitionnées - élagage de partition
# Ne lit que la partition Electronique
electronique = spark.read.parquet("grandes_ventes_partitionne").filter(col("categorie") == "Electronique")

print("Plan de requête avec élagage de partition :")
electronique.explain()

In [ ]:
# Évolution du schéma avec Parquet
# Données originales
donnees_originales = [(1, "A", 100), (2, "B", 200)]
df_original = spark.createDataFrame(donnees_originales, ["id", "code", "valeur"])
df_original.write.mode("overwrite").parquet("test_evolution_schema")

# Nouvelles données avec colonne supplémentaire
nouvelles_donnees = [(3, "C", 300, "nouvelle_info"), (4, "D", 400, "plus_info")]
nouveau_df = spark.createDataFrame(nouvelles_donnees, ["id", "code", "valeur", "extra"])
nouveau_df.write.mode("append").option("mergeSchema", "true").parquet("test_evolution_schema")

# Lire avec schéma fusionné
fusionne = spark.read.option("mergeSchema", "true").parquet("test_evolution_schema")
print("Schéma fusionné :")
fusionne.printSchema()
fusionne.show()

In [ ]:
# Travailler avec plusieurs sources de données
# Exemple : Joindre des données de différents fichiers

# Écrire des données de référence
categories = [("Electronique", "Produits technologiques", 0.1), 
              ("Vetements", "Habillement", 0.05), 
              ("Livres", "Matériel de lecture", 0.0)]
categories_df = spark.createDataFrame(categories, ["categorie", "description", "taux_taxe"])
categories_df.write.mode("overwrite").parquet("categories_ref")

# Lire et joindre
ventes = spark.read.parquet("grandes_ventes_parquet")
categories = spark.read.parquet("categories_ref")

enrichi = ventes.join(broadcast(categories), "categorie")
enrichi = enrichi.withColumn("taxe", col("chiffre_affaires") * col("taux_taxe"))
enrichi = enrichi.withColumn("total", col("chiffre_affaires") + col("taxe"))

print("Données de ventes enrichies :")
enrichi.select("id", "categorie", "chiffre_affaires", "taux_taxe", "taxe", "total").show(10)

In [ ]:
# Nettoyer les fichiers temporaires
import shutil

dossiers_nettoyage = [
    "grandes_ventes_csv", "grandes_ventes_parquet", "grandes_ventes_parquet_gzip",
    "grandes_ventes_orc", "grandes_ventes_partitionne", "test_evolution_schema",
    "categories_ref", "donnees_ventes.parquet", "languages.orc", "languages.parquet", "languages.csv"
]

for d in dossiers_nettoyage:
    if os.path.exists(d):
        shutil.rmtree(d) if os.path.isdir(d) else os.remove(d)
        print(f"Nettoyé : {d}")

### Questions - Exercice 7

**Q7.1** Téléchargez un grand ensemble de données (au moins 1 million de lignes) depuis une source publique (ex: données des taxis de NYC, vues des pages Wikipedia) :
- Chargez-le dans Spark
- Sauvegardez en formats CSV, Parquet et ORC
- Comparez les tailles de fichiers et les temps de lecture/écriture
- Testez les performances des requêtes sur chaque format

**Q7.2** Implémentez un pipeline ETL (Extract, Transform, Load) :
- Lisez les données depuis plusieurs fichiers sources
- Nettoyez et transformez les données (gérez les valeurs nulles, normalisez les valeurs)
- Joignez avec des données de référence
- Écrivez dans des fichiers Parquet partitionnés
- Mesurez et optimisez les performances

**Q7.3** Créez un schéma en étoile d'entrepôt de données :
- Concevez des tables de faits et de dimensions
- Générez des données synthétiques réalistes (10 millions+ de lignes)
- Implémentez des requêtes analytiques courantes
- Optimisez en utilisant le partitionnement, la mise en cache et les jointures broadcast
- Documentez les métriques de performance

In [ ]:
# Vos solutions ici


---

## Résumé

Dans ce TP, vous avez appris :

1. **Architecture Spark** : Comprendre les drivers, executors et cluster managers
2. **RDD** : Créer et manipuler des ensembles de données distribués résilients
3. **Transformations vs Actions** : Évaluation paresseuse et optimisation
4. **DataFrames** : Traitement de données structurées avec schémas
5. **Spark SQL** : Interroger les données avec la syntaxe SQL
6. **Opérations avancées** : Jointures, fonctions fenêtrées, agrégations
7. **Optimisation des performances** : Partitionnement, mise en cache, variables broadcast
8. **Formats de fichiers** : Travailler avec Parquet, ORC et CSV à grande échelle

### Points clés à retenir

- Utilisez les DataFrames plutôt que les RDD quand c'est possible pour une meilleure optimisation
- Mettez en cache les résultats intermédiaires utilisés plusieurs fois
- Utilisez les formats en colonnes (Parquet/ORC) pour les grands ensembles de données
- Partitionnez les données par les colonnes fréquemment filtrées
- Utilisez les jointures broadcast pour les petites tables de référence
- Surveillez les plans de requête avec `explain()` pour identifier les goulots d'étranglement

### Lectures complémentaires

- [Documentation Apache Spark](https://spark.apache.org/docs/latest/)
- [Référence API PySpark](https://spark.apache.org/docs/latest/api/python/)
- [Spark: The Definitive Guide](https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/)
- [Learning Spark, 2nd Edition](https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/)

In [ ]:
# Arrêter la session Spark quand terminé
spark.stop()
sc.stop()
print("Session Spark arrêtée.")